In [130]:
import sys
# !conda install --yes --prefix {sys.prefix} plotly
!pip install country-converter

     |████████████████████████████████| 49 kB 3.8 MB/s eta 0:00:01
  Created wheel for country-converter: filename=country_converter-0.7.3-py3-none-any.whl size=52283 sha256=d8220cb2df65d4c2cdede14ad65436d09f484be48c6ff51ea06a62b2e0cfc5da
  Stored in directory: /Users/ayumis/Library/Caches/pip/wheels/44/5a/d0/9f8289ce8e394f317bb0bc304e6f22ec2b19031b0c92c6f0cb
Successfully built country-converter


In [166]:
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import plotly.express as px
import math
import numpy as np
import country_converter as coco
from scipy import stats
from sklearn.linear_model import LinearRegression

In [15]:
df = pd.read_stata('/Users/ayumis/Desktop/replication_data/output/total.dta')
df = df.dropna(subset=['total_gdp'])
df['iso'] = coco.convert(names = df['countries'], to = 'ISO3')
japanese_names = pd.read_csv('/Users/ayumis/Desktop/interactive_plot/japanese_country_names.csv')
japanese_names.rename(columns={japanese_names.columns[0]: "jp_name", japanese_names.columns[3]: "iso"}, inplace = True)
df = pd.merge(df, japanese_names[['jp_name', 'iso']], on='iso', how = 'left')

In [247]:
reg = LinearRegression()
x = df[['democracy_fh']]
y = df['gdp_growth'].values
w = df['total_gdp'].values
reg.fit(x, y, w) 
line = reg.coef_*df['democracy_fh'] + reg.intercept_
# line = slope*xi+intercept
# x, y, w = df['gdp_growth'],df['democracy_fh'], df['total_gdp']
# reg = LinearRegression()
# # pd.Series(x).array
# reg.fit(df[['gdp_growth', 'democracy_fh']], y, w)
# # slope, intercept, r_value, p_value, std_err = stats.LinearRegression(x,y, w)
# # line = slope*x+intercept

In [204]:
print(line)

0     -1.427232
1     -4.527390
2     -1.969759
3     -1.814752
4     -5.922462
         ...   
165   -0.884704
166   -0.187168
167   -4.449887
168   -3.519839
169   -1.582240
Name: democracy_fh, Length: 170, dtype: float32


### GDP growth rate vs. Democracy Index

In [237]:
fig = px.scatter(df, 
                 x="democracy_fh", 
                 y="gdp_growth",
                 size="total_gdp", 
                 color="democracy_fh",
                 hover_name="countries", 
                 size_max=100)
fig.add_trace(go.Scatter(name='OLS回帰線', 
                         x=df['democracy_fh'], 
                         y=line, 
                         mode='lines',
                         marker=go.Marker(color='rgb(31, 119, 180)')))
fig.update_traces(marker = dict(sizemode='area', 
                                line_width=2, 
                               colorscale='Viridis'), 
                 hovertemplate='国名: %{text}' 
                  +  '<br>民主主義指数: %{x:.1f}' 
                  + '<br>GDP成長率: %{y:.1f}%', 
                  text = df['jp_name'])
fig.update_layout(title = dict(text = '民主的な国ほど経済的ダメージが大きい',
                               x = 0.5),
                  titlefont = dict(size=25),
                  width = 800, 
                  height=800, 
                  hoverlabel = dict(bgcolor = "white", 
                                    font_size = 16),
                  xaxis=dict(title = '民主主義指数',
                             gridcolor = 'white', 
                             gridwidth=2, 
                             titlefont = dict(size=20)
                            ), 
                  yaxis = dict(title = '2020年のGDP成長率',
                               gridcolor = 'white', 
                               gridwidth = 2,
                               range = [-15, 5],
                               titlefont = dict(size=20)
                              ), 
                 paper_bgcolor='rgb(243, 243, 243)', 
                 plot_bgcolor='rgb(243, 243, 243)') 
fig.update_traces(
    marker_coloraxis=None,
    showlegend = False
)
fig.show()
fig.write_html("/Users/ayumis/Desktop/interactive_plot/gdp_japanese.html")

/Users/ayumis/opt/anaconda3/lib/python3.8/site-packages/plotly/graph_objs/_deprecations.py:434: DeprecationWarning:

plotly.graph_objs.Marker is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Marker
  - plotly.graph_objs.histogram.selected.Marker
  - etc.




In [248]:
fig = px.scatter(df, 
                 x="democracy_fh", 
                 y="gdp_growth",
                 size="total_gdp", 
                 color="democracy_fh",
                 size_max=100)
fig.add_trace(go.Scatter(name='OLS', 
                         x=df['democracy_fh'], 
                         y=line, 
                         mode='lines',
                         marker=go.Marker(color='rgb(31, 119, 180)')))
fig.update_traces(marker = dict(sizemode='area', 
                                line_width=2, 
                               colorscale='Viridis'), 
                 hovertemplate='Country: %{text}' 
                  +  '<br>Democracy: %{x:.1f}' 
                  + '<br>GDP Growth Rate: %{y:.1f}%', 
                  text = df['countries'])
fig.update_layout(title = dict(text ='Democratic nations face bigger shocks to GDP in 2020',
                               x = 0.5),
                  titlefont = dict(size=25),
                  width = 800, 
                  height=800, 
                  hoverlabel = dict(bgcolor = "white", 
                                    font_size = 16),
                  xaxis=dict(title = 'Democracy Index (Freedom House)',
                             gridcolor = 'white', 
                             gridwidth=2, 
                             titlefont = dict(size=20)
                            ), 
                  yaxis = dict(title = 'GDP Growth Rate in 2020',
                               gridcolor = 'white', 
                               gridwidth = 2,
                               range = [-15, 5],
                               titlefont = dict(size=20)
                              ), 
                 paper_bgcolor='rgb(243, 243, 243)', 
                 plot_bgcolor='rgb(243, 243, 243)') 
fig.update_traces(
    marker_coloraxis=None,
    showlegend = False
)
fig.show()
fig.write_html("/Users/ayumis/Desktop/interactive_plot/gdp_english.html")

/Users/ayumis/opt/anaconda3/lib/python3.8/site-packages/plotly/graph_objs/_deprecations.py:434: DeprecationWarning:

plotly.graph_objs.Marker is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Marker
  - plotly.graph_objs.histogram.selected.Marker
  - etc.




### Covid-19 Deaths Per Million vs. Democracy Index 

In [242]:
reg = LinearRegression()
x = df[['democracy_fh']]
y = df['total_deaths_per_million'].values
w = df['total_gdp'].values
reg.fit(x, y, w) 
line = reg.coef_*df['democracy_fh'] + reg.intercept_

In [233]:
fig = px.scatter(df, 
                 x="democracy_fh", 
                 y="total_deaths_per_million",
                 size="total_gdp", 
                 color="democracy_fh",
                 hover_name="countries", 
                 size_max=100)
fig.add_trace(go.Scatter(name='OLS回帰線', 
                         x=df['democracy_fh'], 
                         y=line, 
                         mode='lines',
                         marker=go.Marker(color='rgb(31, 119, 180)')))
fig.update_traces(marker = dict(sizemode='area', 
                                line_width=2, 
                               colorscale='Viridis'), 
                 hovertemplate='国名: %{text}' 
                  +  '<br>民主主義指数: %{x:.1f}' 
                  + '<br>コロナ死者数（100万人あたり）: %{y:.1f}', 
                  text = df['jp_name'])
fig.update_layout(title = dict(text='民主的な国ほどコロナ死者数が多い',
                               x = 0.5),
                  titlefont = dict(size=25),
                  width = 800, 
                  height=800, 
                  hoverlabel = dict(bgcolor = "white", 
                                    font_size = 16),
                  xaxis=dict(title = '民主主義指数',
                             gridcolor = 'white', 
                             gridwidth=2, 
                             titlefont = dict(size=20)
                            ), 
                  yaxis = dict(title = '2020年のコロナ死者数（100万人あたり）',
                               gridcolor = 'white', 
                               gridwidth = 2,
                               titlefont = dict(size=20)
                              ), 
                 paper_bgcolor='rgb(243, 243, 243)', 
                 plot_bgcolor='rgb(243, 243, 243)') 
fig.update_traces(
    marker_coloraxis=None,
    showlegend = False
)
fig.show()
fig.write_html("/Users/ayumis/Desktop/interactive_plot/deaths_japanese.html")

In [246]:
fig = px.scatter(df, 
                 x="democracy_fh", 
                 y="total_deaths_per_million",
                 size="total_gdp", 
                 color="democracy_fh",
                 size_max=100)
fig.add_trace(go.Scatter(name='OLS', 
                         x=df['democracy_fh'], 
                         y=line, 
                         mode='lines',
                         marker=go.Marker(color='rgb(31, 119, 180)')))
fig.update_traces(
                  marker = dict(sizemode='area', 
                                line_width=2, 
                               colorscale='Viridis'), 
                 hovertemplate='Country: %{text}' 
                  +  '<br>Democracy: %{x:.1f}' 
                  + '<br>Covid-19 Deaths Per Million: %{y:.1f}', 
                  text = df['countries'])
fig.update_layout(title = dict(text = 'Democratic nations suffer more Covid-19 Deaths',
                               x = 0.5),
                  titlefont = dict(size=25),
                  width = 800, 
                  height=800, 
                  hoverlabel = dict(bgcolor = "white", 
                                    font_size = 16),
                  xaxis=dict(title = 'Democracy Index (Freedom House)',
                             gridcolor = 'white', 
                             gridwidth=2, 
                             titlefont = dict(size=20)
                            ), 
                  yaxis = dict(title = 'Covid-19 Deaths Per Million in 2020',
                               gridcolor = 'white', 
                               gridwidth = 2,
                               titlefont = dict(size=20)
                              ), 
                 paper_bgcolor='rgb(243, 243, 243)', 
                 plot_bgcolor='rgb(243, 243, 243)') 
fig.update_traces(
    marker_coloraxis=None,
    showlegend = False
)
fig.show()
fig.write_html("/Users/ayumis/Desktop/interactive_plot/deaths_english.html")